In [1]:
import pickle
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from keras.datasets import cifar100
from tensorflow.random import set_seed
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import cv2

In [2]:
#convert fine label to course label
(fx, fy), (fxx, fyy) = cifar100.load_data()
(cx, cy), (cxx, cyy) = cifar100.load_data(label_mode='coarse') 

fine_to_coarse = {}
for f,c in zip( fy, cy):
    fine_to_coarse[f[0]] = c[0]

x_train = fx
y_train = fy
x_test = fxx
y_test = fyy

print(len(fine_to_coarse))
print(len(set(fine_to_coarse.values())))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

del fx, fy, fxx, fyy

100
20
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [3]:
x_train = np.array( [ preprocess_input(cv2.resize(x,(224,224))) for x in x_train])
x_test =  np.array( [ preprocess_input(cv2.resize(x,(224,224))) for x in x_test])

print(x_train.shape, x_test.shape)

(50000, 224, 224, 3) (10000, 224, 224, 3)


In [4]:
#get resnet model, use that for embedding 
resnet_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
# remove the output layer
resnet_model.layers.pop()
resnet_embd_model  = Model(inputs=resnet_model.inputs, outputs=resnet_model.layers[-1].output)

x_train = resnet_embd_model.predict(x_train)
x_test = resnet_embd_model.predict(x_test)

print(x_train.shape, x_test.shape)


(50000, 1000) (10000, 1000)


In [5]:
def get_data_positive_class(tx, ty, txx, tyy, wanted_class, fine_to_coarse):
    kept_labels = []
    for k, v, in fine_to_coarse.items():
        if v == wanted_class:
            kept_labels.append(k)
        
    train_data, train_labels = [], []
    test_data,  test_labels =  [], []
    
    for x, y in zip(tx, ty):
        y=y[0]
        if y in kept_labels:
            train_data.append(x)
            
    for x, y  in zip (txx, tyy):
        y=y[0]
        if y in kept_labels:
            test_data.append(x)
    
    return  np.array(train_data), \
            np.repeat(np.array([0,1]).reshape(1,2), len(train_data), axis=0 ),\
            np.array(test_data),\
            np.repeat(np.array([0,1]).reshape(1,2), len(test_data), axis=0 )



def get_data_negative_class (all_data, all_labels, all_test_data, all_test_lables, exclude_classes, fine_to_coarse):
    exclude_labels = []
    for k, v, in fine_to_coarse.items():
        if v in exclude_classes:
            exclude_labels.append(k)
        
    train_data, test_data = [], []
    
    for x, y in zip(all_data, all_labels):
        y = y[0]
        if y not in exclude_labels:
            train_data.append(x)
            
    for x, y  in zip (all_test_data, all_test_lables):
        y = y[0]
        if y not in exclude_labels:
            test_data.append(x)
            
    return  np.array(train_data), \
            np.repeat(np.array([1,0]).reshape(1,2), len(train_data), axis=0 ),\
            np.array(test_data),\
            np.repeat(np.array([1,0]).reshape(1,2), len(test_data), axis=0 )
  

In [25]:
def get_accuracy(data, label, model):
    pp = np.argmax(model.predict(data), axis=1)
    return accuracy_score(label, pp)


def model_baseline(x_train, y_train, batch_size=32, epochs=20, verbose=False):
    input_dim = x_train.shape[1]
    input_layer = Input(shape=(input_dim,))
    
    x  = Dense(128, activation='relu',name='shared')(input_layer)
    
    x    = BatchNormalization()(x)
    y = Dense(y_train.shape[-1], activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=y)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=verbose)
    return model


def model_transfer(x_train, y_train, x_transfer, y_transfer, batch_size=48, epochs=20, verbose=False):
    input_dim =x_train.shape[1]
    input_layer = Input(shape=(input_dim,))
    
    x  = Dense(128,  activation='relu',name='shared')(input_layer)
    
    x    = BatchNormalization()(x)
    y = Dense(y_train.shape[1], activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=y)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    
    #training
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=verbose)
    
    #transfering
    model.fit(x_transfer, y_transfer, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=verbose)

    return model

In [38]:
def do_transfer(x_train, y_train, x_test, y_test, 
                       transfer_class_idx, used_idx, transfer_percent, fine_to_coarse,  
                       train_percent=1.0, batch_size= 48, epochs=20, n_iter=25):
    curr_accuracy = []
    
    
    transfer_data, transfer_labels, transfer_test_data, transfer_test_labels = \
        get_data_positive_class(x_train , y_train, x_test, y_test, transfer_class_idx, fine_to_coarse)

    #negative data are all classes that's no in transfer class and used class
    train_neg_data, train_neg_labels, test_neg_data, test_neg_label \
                = get_data_negative_class(x_train , y_train, x_test, y_test, used_idx + [transfer_class_idx], fine_to_coarse)
        
    
    
    transfer_full_data = np.vstack([transfer_data, train_neg_data])
    transfer_full_labels = np.vstack([transfer_labels, train_neg_labels])

    transfer_full_test  = np.vstack([transfer_test_data, test_neg_data])
    transfer_full_test_lables = np.vstack([transfer_test_labels, test_neg_label])
    

    #for other in wanted_task_idx:
    for other in used_idx:
        
        train_pos_data, train_pos_labels, test_pos_data, test_pos_labels = \
            get_data_positive_class(x_train , y_train, x_test, y_test, other, fine_to_coarse)
        
        
        train_data = np.vstack([train_pos_data, train_neg_data])
        train_labels = np.vstack([train_pos_labels, train_neg_labels])

        test_data = np.vstack([test_pos_data, test_neg_data])
        test_labels = np.vstack([test_pos_labels, test_neg_label])

        transfer_accuracy = []

        for i in range(n_iter):
            if i % 5 == 0:
                print(i,  transfer_percent, coarse_names[other], np.average(transfer_accuracy) )
            

            #base_case data
            total = train_data.shape[0]
            train_size = int( total * train_percent)


            idx = np.random.choice(total, train_size, replace=False)
            d0 = train_data[idx]
            l0 = train_labels[idx]


            #transfer data!!!
            total = transfer_full_data.shape[0]
            transfer_size = int( total * transfer_percent)
            
            idx = np.random.choice(total, transfer_size, replace=False)
            d1 = transfer_full_data[idx]
            l1 = transfer_full_labels[idx]
                        
#             print(d0.shape, d1.shape)
            
            model = model_transfer(d0, l0, d1, l1, batch_size=batch_size, epochs=epochs)

            p = model.predict(transfer_full_test).argmax(axis=1)
            t = transfer_full_test_lables.argmax(axis=1)

            f1 = f1_score(p, t, labels=[1])            
            transfer_accuracy.append(f1)

        curr_accuracy.append( (coarse_names[other], transfer_accuracy ) )
    
    return curr_accuracy

In [8]:
def do_baseline(x_train, y_train, x_test, y_text, transfer_class, used_idx, use_percent, fine_to_coarse,  
                train_percent=1.0, batch_size= 48, epochs=20, n_iter=25):
    
    transfer_data, transfer_labels, transfer_test_data, transfer_test_labels = \
        get_data_positive_class(x_train , y_train, x_test, y_test, transfer_class, fine_to_coarse)

    #negative data are all classes that's no in transfer class and used class
    train_neg_data, train_neg_labels, test_neg_data, test_neg_label \
                = get_data_negative_class(x_train , y_train, x_test, y_test, used_idx + [transfer_class], fine_to_coarse)
        
    
    
    transfer_full_data = np.vstack([transfer_data, train_neg_data])
    transfer_full_labels = np.vstack([transfer_labels, train_neg_labels])

    transfer_full_test  = np.vstack([transfer_test_data, test_neg_data])
    transfer_full_test_lables = np.vstack([transfer_test_labels, test_neg_label])
    
    accuracies  = []
    for i in range(n_iter):
        if i % 5 == 0:
            print(i,  use_percent, np.average(accuracies) )

        total = transfer_full_data.shape[0]
        transfer_size = int( total * use_percent)

        idx = np.random.choice(total, transfer_size, replace=False)
        d = transfer_full_data[idx]
        l = transfer_full_labels[idx]
        model = model_baseline(d, l, batch_size=batch_size, epochs=epochs)
        
        p = model.predict(transfer_full_test).argmax(axis=1)
        t = transfer_full_test_lables.argmax(axis=1)
        
        f1  = f1_score(p ,t, labels=[1])
        accuracies.append(f1)
    return accuracies
                        

    

In [9]:
wanted = 8 #'large carnivores'

coarse_names = [
'aquatic_mammals',  'fish', 'flowers', 'food_containers', 
'fruit_and_vegetables', 'household_electrical_devices', 
'household_furniture', 'insects', 'large_carnivores',
'large_man-made_outdoor_things', 'large_natural_outdoor_scenes',
'large_omnivores_and_herbivores', 'medium-sized_mammals',
'non-insect_invertebrates', 'people', 'reptiles',
'small mammals', 'trees', 'vehicles_1','vehicles_2'
]


used_set  = ['small mammals',  'vehicles_1', ]
used_idx = [i for i, x in enumerate(coarse_names) if x  in used_set ]


In [12]:
seed = 9876
np.random.seed(seed)
set_seed(seed)

#measure baseline (non_transfer) accuracy 
ranges = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
n_iter = 25
baseline_acc = {}
for percentage in ranges:
    baseline_acc[percentage] = do_baseline(x_train, y_train, x_test, y_test, wanted, used_idx, percentage, fine_to_coarse, n_iter=n_iter)
    print('Percentage %s : accuracy %s' %(percentage, np.average(baseline_acc[percentage])))

0 0.1 nan
5 0.1 0.6197617460178546
10 0.1 0.6244536298960021
15 0.1 0.6227700096703949
20 0.1 0.620197472036649
Percentage 0.1 : accuracy 0.6208906403066979
0 0.2 nan
5 0.2 0.6468298095722014
10 0.2 0.640583293867922
15 0.2 0.6421785603921671
20 0.2 0.6449555998486084
Percentage 0.2 : accuracy 0.6456671853576239
0 0.3 nan
5 0.3 0.6670212722384782
10 0.3 0.6641367479491318
15 0.3 0.6628303198691989
20 0.3 0.6629155369549433
Percentage 0.3 : accuracy 0.6626568296923332
0 0.4 nan
5 0.4 0.670400679852899
10 0.4 0.6702919855831613
15 0.4 0.6657883962177366
20 0.4 0.6648480220982453
Percentage 0.4 : accuracy 0.6663950002258191
0 0.5 nan
5 0.5 0.6751580717305992
10 0.5 0.6725167406195655
15 0.5 0.6738519400015626
20 0.5 0.676659263874361
Percentage 0.5 : accuracy 0.678557516392326
0 0.6 nan
5 0.6 0.6882917164788004
10 0.6 0.6861566233244129
15 0.6 0.6867393569142843
20 0.6 0.6854714174902556
Percentage 0.6 : accuracy 0.6861142300826293
0 0.7 nan
5 0.7 0.6919288979770658
10 0.7 0.6923305240001

In [ ]:
seed = 9876
np.random.seed(seed)
set_seed(seed)

ranges = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
n_iter = 25
transfer_acc = {}
for percentage in ranges:
    res = do_transfer(x_train, y_train, x_test, y_test, 
                       wanted, used_idx, percentage, fine_to_coarse, n_iter=n_iter)
    tmp = res
    tmp = sorted(tmp, key=lambda x : np.average(x[1]))
    transfer_acc[percentage] = res
    print(percentage)
    for t in tmp:
        print (t[0], np.average(t[1]))
    


0 0.1 small mammals nan


In [37]:
transfer_acc

{0.1: [('small mammals', [0.614572333685322]),
  ('vehicles_1', [0.6019007391763463])],
 0.2: [('small mammals', [0.6372745490981964]),
  ('vehicles_1', [0.6335797254487857])]}

In [14]:
with open ('./baseline_acc.p', 'wb') as handle:
    pickle.dump(baseline_acc, handle)

In [ ]:
with open('./tranfer_acc.p', 'wb') as handle:
    pickle.dume(transfer_acc, handle)